# Analysis of COVID-19 infections data in Italy

## Authors:
> **Jacopo Bartoli**  
> Università degli Studi di Firenze - Scuola di Ingegneria  
> jacopo.bartoli@stud.unifi.it  

> **Jason Ravagli**  
> Università degli Studi di Firenze - Scuola di Ingegneria  
> jason.ravagli@stud.unifi.it

---

## Outline
1. Introduction
2. Section 2
3. Section 3
4. ...

---

## 1. Introduction
We are all aware of the situation that the COVID-19 pandemic has caused in Italy during 2020. Starting from March 2020, the Italian Civil Protection (ICP) has begun to gather and make available daily data about contagious (such as the number of new positives, hospitalized infected, deaths, etc..) in our country to inform the population.

In addition to using the usual communication channels, the ICP created a [GitHub repository](https://github.com/pcm-dpc/COVID-19) to publish all the data and make them usable to research purposes. The main contributor to this repository is Umberto Rosini (https://github.com/umbros), an employee of the ICP.

The repository contains various kinds of data regarding the emergency and the infection, but in this notebook we focus on the analysis of two data files:

- https://github.com/pcm-dpc/COVID-19/blob/master/dati-json/dpc-covid19-ita-province.json
- https://github.com/pcm-dpc/COVID-19/blob/master/dati-json/dpc-covid19-ita-regioni.json

They contain the data about contagion day by day since 24th February 2020 and, at the time of writing (21st August 2020), they are updated every day at 18:30.  
The first file contains data divided by province, while the second one contains data divided by region.

**Summary**

### Setup environment

Importing required libraries:

In [117]:
import json
import pandas as pd

Define a function that allows to display multiple objects (e.g. dataframes) side by side:

In [118]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## 2. Loading data

### Loading data as list of dictionaries

Inspecting the JSON files, we can observe that in each of them data are stored as an array of JSON objects. Therefore, we can load data in a list of Python dictionaries to take a first look at their structure.

In [119]:
with open("data/dpc-covid19-ita-regioni.json") as f:
    list_dict_reg = json.load(f)

In [120]:
with open("data/dpc-covid19-ita-province.json") as f:
    list_dict_prov = json.load(f)

In [121]:
list_dict_reg[0]

{'data': '2020-02-24T18:00:00',
 'stato': 'ITA',
 'codice_regione': 13,
 'denominazione_regione': 'Abruzzo',
 'lat': 42.35122196,
 'long': 13.39843823,
 'ricoverati_con_sintomi': 0,
 'terapia_intensiva': 0,
 'totale_ospedalizzati': 0,
 'isolamento_domiciliare': 0,
 'totale_positivi': 0,
 'variazione_totale_positivi': 0,
 'nuovi_positivi': 0,
 'dimessi_guariti': 0,
 'deceduti': 0,
 'casi_da_sospetto_diagnostico': None,
 'casi_da_screening': None,
 'totale_casi': 0,
 'tamponi': 5,
 'casi_testati': None,
 'note': None}

In [122]:
list_dict_prov[0]

{'data': '2020-02-24T18:00:00',
 'stato': 'ITA',
 'codice_regione': 13,
 'denominazione_regione': 'Abruzzo',
 'codice_provincia': 66,
 'denominazione_provincia': "L'Aquila",
 'sigla_provincia': 'AQ',
 'lat': 42.35122196,
 'long': 13.39843823,
 'totale_casi': 0,
 'note': None}

Each data object contains information about the cases of COVID-19 in a particular day for each region/province.
Data about regions contain exhaustive information about the detected cases, and a full explanation of each data field can be found [here](https://github.com/pcm-dpc/COVID-19/blob/master/dati-andamento-covid19-italia.md).

However it is not clear what the field *totale_casi* in the province data is about. In particular, from the documentation it is not clear if it refers to the currently positive cases in that province or to the overall detected cases.

### Loading data as Pandas dataframe

In [123]:
df_reg = pd.DataFrame(list_dict_reg)

In [124]:
df_prov = pd.DataFrame(list_dict_prov)

---

Let us take an overview at the region data.

In [125]:
df_reg.shape

(3738, 21)

In [126]:
df_reg.dtypes

data                             object
stato                            object
codice_regione                    int64
denominazione_regione            object
lat                             float64
long                            float64
ricoverati_con_sintomi            int64
terapia_intensiva                 int64
totale_ospedalizzati              int64
isolamento_domiciliare            int64
totale_positivi                   int64
variazione_totale_positivi        int64
nuovi_positivi                    int64
dimessi_guariti                   int64
deceduti                          int64
casi_da_sospetto_diagnostico    float64
casi_da_screening               float64
totale_casi                       int64
tamponi                           int64
casi_testati                    float64
note                             object
dtype: object

In [127]:
df_reg.head()

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,None
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,None
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,None
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,None
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,None


In [128]:
df_reg.describe()

,codice_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
count,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,3738.000000,1176.000000,1176.000000,3738.000000,3.738000e+03,2583.000000
mean,11.857143,43.046293,12.225955,444.334136,50.146602,494.480738,1557.240770,2051.721509,4.050027,68.313002,5435.201980,1208.643125,10614.001701,1097.881803,8695.566613,1.607540e+05,137174.008904
std,6.274638,2.489114,2.659303,1323.771538,140.420552,1459.140050,3563.133122,4776.091041,185.709468,199.965627,11458.321257,2992.315037,19311.460927,1798.906944,17203.130676,2.438840e+05,145445.090715
min,1.000000,38.115697,7.320149,0.000000,0.000000,0.000000,0.000000,0.000000,-5339.000000,-229.000000,0.000000,0.000000,400.000000,0.000000,0.000000,0.000000e+00,3482.000000
25%,7.000000,41.125596,11.121231,9.000000,0.000000,10.000000,72.000000,87.250000,-12.000000,1.000000,245.250000,64.000000,1433.000000,17.000000,1100.000000,1.589075e+04,43673.500000
50%,12.000000,43.616760,12.388247,56.000000,5.000000,63.000000,320.000000,398.000000,0.000000,9.000000,1351.000000,291.000000,3348.000000,191.500000,3154.500000,7.786100e+04,83144.000000
75%,17.000000,45.434905,13.768136,255.000000,36.000000,294.000000,1313.750000,1648.250000,14.000000,48.000000,4404.000000,870.000000,8614.500000,1579.000000,7549.500000,1.888848e+05,182034.500000
max,22.000000,46.499335,16.867367,12077.000000,1381.000000,13328.000000,30382.000000,37307.000000,1950.000000,3251.000000,75398.000000,16844.000000,90487.000000,7485.000000,97564.000000,1.432476e+06,855372.000000


---

Now let us do the same thing for the province data.

In [129]:
df_prov.shape

(23960, 11)

In [130]:
df_prov.dtypes

data                        object
stato                       object
codice_regione               int64
denominazione_regione       object
codice_provincia             int64
denominazione_provincia     object
sigla_provincia             object
lat                        float64
long                       float64
totale_casi                  int64
note                        object
dtype: object

In [131]:
df_prov.head()

,data,stato,codice_regione,denominazione_regione,codice_provincia,denominazione_provincia,sigla_provincia,lat,long,totale_casi,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,66,L'Aquila,AQ,42.351222,13.398438,0,None
1,2020-02-24T18:00:00,ITA,13,Abruzzo,67,Teramo,TE,42.658918,13.704400,0,None
2,2020-02-24T18:00:00,ITA,13,Abruzzo,68,Pescara,PE,42.464584,14.213648,0,None
3,2020-02-24T18:00:00,ITA,13,Abruzzo,69,Chieti,CH,42.351032,14.167546,0,None
4,2020-02-24T18:00:00,ITA,13,Abruzzo,979,In fase di definizione/aggiornamento,None,NaN,NaN,0,None


In [132]:
df_prov.describe()

,codice_regione,codice_provincia,lat,long,totale_casi
count,23960.000000,23960.000000,19046.000000,19046.000000,23960.000000
mean,10.499499,240.971536,42.908522,12.096599,1356.585935
std,6.053899,369.370866,2.631578,2.660616,2748.219583
min,1.000000,1.000000,36.925092,7.320149,0.000000
25%,5.000000,34.000000,40.914047,9.828190,59.000000
50%,9.000000,68.000000,43.715532,12.104734,424.000000
75%,16.000000,101.000000,45.185093,14.167546,1367.000000
max,22.000000,999.000000,46.499335,18.171897,25297.000000


### Cleaning and preparing data

From the above tables we can observe that there are some useless data in both dataframes.
First of all we can drop the column `stato` from both, since the data all refer to the Italy (we can verify that `ITA` is the unique value).

In [133]:
df_reg['stato'].unique(), df_prov['stato'].unique()

(array(['ITA'], dtype=object), array(['ITA'], dtype=object))

In [134]:
df_reg = df_reg.drop(columns=["stato"])
df_prov = df_prov.drop(columns=["stato"])

Also the field `note` can be removed from the regions dataframe since it contains not useful plain textual data.

In [135]:
df_reg["note"].unique()

array([None, 'pd-IT-0002', 'pd-IT-0004', 'nd-IT-0001', 'nd-IT-0003',
       'nd-IT-0005', 'nd-IT-0008', 'nd-IT-0013', 'nd-IT-0011',
       'pd-IT-0006', 'pd-IT-0008', 'pd-IT-0010', 'dc-IT-0002',
       'dc-IT-0004', 'dc-IT-0006', 'pd-IT-0012', 'pd-IT-0014',
       'pd-IT-0016', 'dc-IT-0008', 'dc-IT-0010', 'dc-IT-0012',
       'dc-IT-0014', 'dc-IT-0016', 'dc-IT-0018', 'dc-IT-0020',
       'dc-IT-0022', 'dc-IT-0024', 'dc-IT-0026', 'dc-IT-0028',
       'dc-IT-0030', 'pd-IT-0018', 'dc-IT-0032', 'pd-IT-0020',
       'dc-IT-0034', 'dc-IT-0036', 'nd-IT-0015', 'pd-IT-0022',
       'dc-IT-0037', 'dc-IT-0038', 'dc-IT-0040', 'dc-IT-0042',
       'dc-IT-0046', 'dc-IT-0044', 'dc-IT-0048', 'dc-IT-0050',
       'dc-IT-0052', 'dc-IT-0054', 'dc-IT-0056', 'dc-EN-0058',
       'dc-IT-0062', 'dc-IT-0060', 'dc-IT-0064', 'dc-IT-0066',
       'dc-IT-0068', 'dc-IT-0070',
       'n.3 decessi precedentemente segnalati, da una revisione non risultano classificabili come COVID-19 positivi',
       'NEI CASI POSIT

In [136]:
df_reg = df_reg.drop(columns=["note"])

Looking at the column types, we can see that the data under the columns `data` are stored as `object` in both dataframes. It is more appropriate to use the pandas `Timestamp` type, to facilitate queries.  
According to the documentation and to the size of data, for each day and region (for each day and province for the provinces dataframe) there is a unique entry, so for simplicity we can keep only the date without the time normalizing the column.

In [137]:
df_reg["data"] = pd.DatetimeIndex(df_reg["data"]).normalize()
df_prov["data"] = pd.DatetimeIndex(df_prov["data"]).normalize()

Finally, it would be useful for the next analysis to modify the indexes to use a multi-index with the date and regions for the regions data and a multi-index with the date and provinces for the provinces ones.

In [139]:
df_reg.set_index(["data", "denominazione_regione"], inplace=True)
df_prov.set_index(["data", "denominazione_regione", "sigla_provincia"], inplace=True)

Let us look at the obtained data:

## !!! Guarda il df delle province: NaN nell'index, droppare i dati non assegnati?

In [145]:
df_reg.head()

codice_regione        lat       long  \
data       denominazione_regione                                         
2020-02-24 Abruzzo                            13  42.351222  13.398438   
           Basilicata                         17  40.639471  15.805148   
           Calabria                           18  38.905976  16.594402   
           Campania                           15  40.839566  14.250850   
           Emilia-Romagna                      8  44.494367  11.341721   

                                  ricoverati_con_sintomi  terapia_intensiva  \
data       denominazione_regione                                              
2020-02-24 Abruzzo                                     0                  0   
           Basilicata                                  0                  0   
           Calabria                                    0                  0   
           Campania                                    0                  0   
           Emilia-Romagna                             10                  2   

                                  totale_ospedalizzati  \
data       denominazione_regione                         
2020-02-24 Abruzzo                                   0   
           Basilicata                                0   
           Calabria                                  0   
           Campania                                  0   
           Emilia-Romagna                           12   

                                  isolamento_domiciliare  totale_positivi  \
data       denominazione_regione                                            
2020-02-24 Abruzzo                                     0                0   
           Basilicata                                  0                0   
           Calabria                                    0                0   
           Campania                                    0                0   
           Emilia-Romagna                              6               18   

                                  variazione_totale_positivi  nuovi_positivi  \
data       denominazione_regione                                               
2020-02-24 Abruzzo                                         0               0   
           Basilicata                                      0               0   
           Calabria                                        0               0   
           Campania                                        0               0   
           Emilia-Romagna                                  0              18   

                                  dimessi_guariti  deceduti  \
data       denominazione_regione                              
2020-02-24 Abruzzo                              0         0   
           Basilicata                           0         0   
           Calabria                             0         0   
           Campania                             0         0   
           Emilia-Romagna                       0         0   

                                  casi_da_sospetto_diagnostico  \
data       denominazione_regione                                 
2020-02-24 Abruzzo                                         NaN   
           Basilicata                                      NaN   
           Calabria                                        NaN   
           Campania                                        NaN   
           Emilia-Romagna                                  NaN   

                                  casi_da_screening  totale_casi  tamponi  \
data       denominazione_regione                                            
2020-02-24 Abruzzo                              NaN            0        5   
           Basilicata                           NaN            0        0   
           Calabria                             NaN            0        1   
           Campania                             NaN            0       10   
           Emilia-Romagna                       NaN           18      148   

        

In [143]:
df_prov.head()

codice_regione  \
data       denominazione_regione sigla_provincia                   
2020-02-24 Abruzzo               AQ                           13   
                                 TE                           13   
                                 PE                           13   
                                 CH                           13   
                                 NaN                          13   

                                                  codice_provincia  \
data       denominazione_regione sigla_provincia                     
2020-02-24 Abruzzo               AQ                             66   
                                 TE                             67   
                                 PE                             68   
                                 CH                             69   
                                 NaN                           979   

                                                               denominazione_provincia  \
data       denominazione_regione sigla_provincia                                         
2020-02-24 Abruzzo               AQ                                           L'Aquila   
                                 TE                                             Teramo   
                                 PE                                            Pescara   
                                 CH                                             Chieti   
                                 NaN              In fase di definizione/aggiornamento   

                                                        lat       long  \
data       denominazione_regione sigla_provincia                         
2020-02-24 Abruzzo               AQ               42.351222  13.398438   
                                 TE               42.658918  13.704400   
                                 PE               42.464584  14.213648   
                                 CH               42.351032  14.167546   
                                 NaN                    NaN        NaN   

                                                  totale_casi  note  
data       denominazione_regione sigla_provincia                     
2020-02-24 Abruzzo               AQ                         0  None  
                                 TE                         0  None  
                                 PE                         0  None  
                                 CH                         0  None  
                                 NaN                        0  None

## 3. Analysis of national data

As a first step, let us analyze the data from a national point of view.

The simplest statistics to extract are those referring to the total number of deaths, healed and infected since the beginning of the pandemic.